# Projeto 2 - Classificando Automático de Sentimento 

## Por: Beatriz Alexandre e Vitória Almeida

#### Marca escolhida: Lindt

Fomos contratadas pela empresa Lindt, pára analisar os tweets feitos por seus clientessobre sua empresa. Para isso criaremos um programa que irá analisae as mensagens disponiveis como "relevante" ou "irrelevante". Os twers que forem negativos irão disparar um foco na área de marketing.

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl  
import seaborn as sns
import numpy as np
import itertools
from scipy import stats
from emoji import UNICODE_EMOJI

#### Colocando os dados em um DataFrame

In [2]:
tw = pd.read_excel('tweets_lindt.xlsx', sheet_name = 0)

In [3]:
tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1,0
1,@elgliko hear that pitter patter in the back g...,0,NaN
2,rt @statutory_boy: the guy was addicted to dru...,0,NaN
3,"@andreyasasylum @yourmomsuckstho hey, can you ...",1,1
4,great job by @noticgroup gms. come and see the...,1,1


#### Limpando os tweets

LIMPAR TWEETS

In [4]:
import string
ponto = string.punctuation

#lista para depoi tirar os pontos
po = ["\n", "\t"]

data_limpo = []

#dar espaço nos emojis e limpar os pontos
for tweet in tw["Treinamento"]:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        elif palavra in po:
            t += " "
        elif palavra not in ponto:
            t += palavra
    data_limpo.append(t)

tw_li =  pd.DataFrame()
tw_li['Treinamento'] = data_limpo
tw_li['Relevante'] = tw['Relevante']

tw_li.head()



,Treinamento,Relevante
0,lindt chocolate is so overrated,1
1,elgliko hear that pitter patter in the back gr...,0
2,rt statutoryboy the guy was addicted to drugs ...,0
3,andreyasasylum yourmomsuckstho hey can you ple...,1
4,great job by noticgroup gms come and see them ...,1


In [13]:
# tirar os @ e os #
data_limpo = []
x= ' '
for tweet in tw_li['Treinamento']:
    limp = []
    spli = tweet.split(" ")
    for palavra in spli:
        if palavra in UNICODE_EMOJI:
            limp.append(palavra)
        elif len(palavra) > 2 and palavra[0]!='@' and palavra[0] != '#' and palavra[:4] != "http":
            limp.append(palavra)
    data_limpo.append(x.join(limp))
    
tw_limpo =  pd.DataFrame()
tw_limpo['Treinamento'] = data_limpo
tw_limpo['Relevante'] = tw['Relevante']
tw_limpo['Treinamento'] = tw_limpo['Treinamento'].str.replace('gorgeous\xa0i', 'banana')
#tw_limpo['Treinamento'] = tw_limpo['Treinamento'].str.replace(" ' ' ", " ")

tw_limpo.head()

,Treinamento,Relevante
0,lindt chocolate overrated,1
1,elgliko hear that pitter patter the back groun...,0
2,statutoryboy the guy was addicted drugs his li...,0
3,andreyasasylum yourmomsuckstho hey can you ple...,1
4,great job noticgroup gms come and see them and...,1


tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('@', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace(':', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('_', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('%', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('&', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace(',', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('#', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace(';', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('.', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('$', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('(', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace(')', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('*', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('/', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('?', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('+', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('-', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace(']', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('[', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('=', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('^', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('!', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('"', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace('\n', '')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace("'", "")
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace("’",'')
tw_limpo.Treinamento = tw_limpo.Treinamento.str.replace("''",'')

tw.head()

Dar espaço nos emojis:

Lista de todas as palavras dos tweets:

In [14]:
pala = []

for i in tw_limpo['Treinamento']:
    frase = i.split()
    for p in frase:
        if p not in pala:
            pala.append(p)

Contar quantas vezes aparace o 1(Relevante) e o 0(não relevante):

In [15]:
#contador
Rel = 0
Irre = 0 

for l in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][l].split(" ")
    for i in linha:
        if tw_limpo["Treinamento"][l] == 1:
            Rel += 1
        else:
            Irre += 1

Dicionários para calcular a frequencia de relevantes e irrelevantes:

In [16]:
#adicionou as palavras no dicionário de frequencia
r_freq = {}
ir_freq = {}

for palavras in pala:
    r_freq[palavras] = 1
    ir_freq[palavras] = 1

for g in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][g].split(" ")
    for v in linha:
        if tw_limpo["Relevante"][g] == 0:
            ir_freq[v]+=1
        else:
            r_freq[v]+= 1

A probabilidade de ser relevante da cada palavra

In [17]:
r_prob = {}
ir_prob = {}

#pala é a lista de palavras dos tweets

for palavra in pala:
    r_prob[palavra] = r_freq[palavra]/(len(palavra)+Rel)
    ir_prob[palavra] = ir_freq[palavra]/(len(palavra)+Irre)
    

P(relevante) e P(irrelevante)

In [18]:
irrelevante = 0
relevante = 0 
for m in tw_limpo['Relevante']:
    if m == 0:
        irrelevante += 1
    else:
        relevante += 1 

Pi = irrelevante/len(tw_limpo['Relevante'])
Pr = relevante/len(tw_limpo['Relevante'])


Teoria de Naive - Bayes

# TESTE

In [19]:
g = pd.read_excel('tweets_lindt.xlsx', sheet_name = 1)

In [20]:
g.head()

,Teste,Relevante,Positivo/Negativo
0,rt @thembarumbu: dear ladies\nif you catch ur ...,0,NaN
1,rt @dailytelegraph: one of the most notorious ...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,@hloni_maniers i'm craving lindt lindor 😫,1,1
4,rt @lutho__m: if you're reading this tweet htt...,0,NaN


In [21]:
g.Teste = g.Teste.str.replace('@', '')
g.Teste = g.Teste.str.replace(':', '')
g.Teste = g.Teste.str.replace('-', '')
g.Teste = g.Teste.str.replace('_', '')
g.Teste = g.Teste.str.replace(',', '')
g.Teste = g.Teste.str.replace('!', '')
g.Teste = g.Teste.str.replace('(', '')
g.Teste = g.Teste.str.replace(')', '')
g.Teste = g.Teste.str.replace('=', '')
g.Teste = g.Teste.str.replace('*', '')
g.Teste = g.Teste.str.replace('#', '')

g.head()

,Teste,Relevante,Positivo/Negativo
0,rt thembarumbu dear ladies\nif you catch ur bo...,0,NaN
1,rt dailytelegraph one of the most notorious ba...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,hlonimaniers i'm craving lindt lindor 😫,1,1
4,rt luthom if you're reading this tweet https//...,0,NaN


In [22]:
g_espaco_emoji = []
data = g["Teste"]
for tweet in data:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        else:
            t += palavra
    g_espaco_emoji.append(t)

g_limpo =  pd.DataFrame()
g_limpo['Teste'] = g_espaco_emoji
g_limpo['Relevante'] = g['Relevante']

g_limpo.head()

,Teste,Relevante
0,rt thembarumbu dear ladies\nif you catch ur bo...,0
1,rt dailytelegraph one of the most notorious ba...,1
2,commercial law's bitch ...is what i am right n...,0
3,hlonimaniers i'm craving lindt lindor 😫,1
4,rt luthom if you're reading this tweet https//...,0


In [24]:
lista_teste = []
j = []
k = []
for f in g_limpo['Teste']:
    comR = 1
    comIr = 1
    x = f.split(' ')
    for palavra in x:
        if palavra in ir_prob:
            comIr *= ir_prob[palavra]
        else:
            comIr *= 1/(len(pala)+Rel)
        if palavra in r_prob:
            comR *= r_prob[palavra]
        else:
            comR *= 1/(len(pala)+Rel)
    Rp = comR*Pr
    Irp = comIr*Pi
    
    j.append(Rp)
    k.append(Irp)
    
    if Rp >= Irp:
        lista_teste.append(1)
    else:
        lista_teste.append(0)
        
min(k)
g["lista teste"] = lista_teste

In [25]:
uu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
uz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
zz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]
zu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]

posV = (len(uu)/len(g['Relevante']))*100
posF = (len(uz)/len(g['Relevante']))*100
negV = (len(zz)/len(g['Relevante']))*100
negF = (len(zu)/len(g['Relevante']))*100

print('Positivos verdadeiros = {:.3f}%'.format(posV))
print('Positivos falsos = {:.3f}%'.format(posF))
print('Negativos verdadeiros = {:.3f}%'.format(negV))
print('Negativos falsos = {:.3f}%'.format(negF))

Positivos verdadeiros = 47.000%
Positivos falsos = 52.500%
Negativos verdadeiros = 0.500%
Negativos falsos = 0.000%


P(Frase|relevante) = P(palavra|relevante) * P(outra_paravra|relevante)
